##1 — Imports & settings

In [ ]:
# 1_imports_and_settings.py
# Purpose: import libs, set plotting style and random seed for reproducibility.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix,\
 roc_auc_score, roc_curve, accuracy_score
from imblearn.over_sampling import SMOTE

# Global plotting settings
sns.set(style="whitegrid", context="notebook")
plt.rcParams['figure.dpi'] = 100

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


### centralizing imports and settings makes the notebook easier to read and ensures reproducible results.

##2 — Load data & initial overview
1. load_data_and_overview.py
2. Load dataset and perform quick checks (head, info,descriptive stats).




In [ ]:
df = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')  # update path if necessary

In [ ]:
# Quick peek
display(df.head())

In [ ]:
# Structure and types
print("===== INFO =====")
df.info()

In [ ]:
# Numerical summary
print("\n===== DESCRIBE (numeric) =====")
display(df.describe(include=[np.number]).T)

In [ ]:
# Check class balance
print("\nStroke value counts:")
display(df['stroke'].value_counts(normalize=False))
display(df['stroke'].value_counts(normalize=True))

1. df.info() tells you types and non-null counts — useful to detect missing values.

2. describe() provides mean, std, quartiles for numeric columns (age, bmi, avg_glucose_level).

3. Check stroke value counts to see class imbalance (likely very imbalanced — few positives).

##3 — Helper: add_percent for countplots


In [ ]:
# 3_plot_helpers.py
# A reusable function to show percentage labels above bars.
def add_percent(ax, df, x, hue=None):
    """
    Annotate bars with percentage of total rows (not percentage
    inside each group).
    ax : matplotlib axis containing bars from seaborn.countplot
    df : original dataframe
    x : x column name (string)
    hue: optional hue column name (string) - not used for grouping
     calculation here
    """
    total = len(df)
    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.text(
                p.get_x() + p.get_width() / 2,
                height + total * 0.005,
                f'{height/total*100:.1f}%',
                ha='center',
                va='bottom',
                fontsize=9
            )


###This function annotates bars with percent of whole dataset. If you want percent within each category (e.g., percent of males that had stroke), we can modify it — tell me and I'll provide that variant.

##4 — Exploratory plots & interpretation

* .  Visual EDA: categorical counts and numeric boxplots vs
stroke.

In [ ]:
# Gender vs stroke
plt.figure(figsize=(6,4))
ax = sns.countplot(x="gender", hue="stroke", data=df)
add_percent(ax, df, "gender", "stroke")
plt.title("Gender vs Stroke (%)")
plt.show()

###Gender vs Stroke: If stroke counts are much higher for one gender proportionally, that suggests a gender association. But always compare proportions (e.g., stroke rate among males vs females) — absolute bar heights can be misleading if group sizes differ.

In [ ]:
# Age vs stroke (boxplot)
plt.figure(figsize=(6,4))
sns.boxplot(x="stroke", y="age", data=df)
plt.title("Age distribution by Stroke (0 = no, 1 = yes)")
plt.show()

###Age boxplot: Likely shows that median age is higher for stroke patients. Look at median, IQR, and outliers. If older ages cluster in stroke=1, age is an important predictor.

In [ ]:
# Hypertension vs stroke
plt.figure(figsize=(6,4))
ax = sns.countplot(x="hypertension", hue="stroke", data=df)
add_percent(ax, df, "hypertension", "stroke")
plt.title("Hypertension vs Stroke (%)")
plt.show()

In [ ]:
# Heart disease vs stroke
plt.figure(figsize=(6,4))
ax = sns.countplot(x="heart_disease", hue="stroke", data=df)
add_percent(ax, df, "heart_disease", "stroke")
plt.title("Heart Disease vs Stroke (%)")
plt.show()

###Hypertension & Heart disease: Expect higher proportion of stroke among people with hypertension or heart disease. These are known risk factors.

In [ ]:
# Ever married vs stroke
plt.figure(figsize=(6,4))
ax = sns.countplot(x="ever_married", hue="stroke", data=df)
add_percent(ax, df, "ever_married", "stroke")
plt.title("Ever Married vs Stroke (%)")
plt.show()

###Ever married: Marriage may correlate with age and socioeconomic factors — interpret cautiously.

In [ ]:
# Work type vs stroke
plt.figure(figsize=(10,4))
ax = sns.countplot(x="work_type", hue="stroke", data=df)
add_percent(ax, df, "work_type", "stroke")
plt.xticks(rotation=30)
plt.title("Work Type vs Stroke (%)")
plt.show()

###Work type: Inspect which job categories have relatively higher stroke proportions. Small categories with high percentages might be noisy (small sample size).

In [ ]:
# Residence type vs stroke
plt.figure(figsize=(6,4))
ax = sns.countplot(x="Residence_type", hue="stroke", data=df)
add_percent(ax, df, "Residence_type", "stroke")
plt.title("Residence Type vs Stroke (%)")
plt.show()

###Residence type: Urban vs rural differences — consider access to healthcare confounding

In [ ]:
# avg_glucose_level vs stroke (boxplot)
plt.figure(figsize=(6,4))
sns.boxplot(x="stroke", y="avg_glucose_level", data=df)
plt.title("Average Glucose Level by Stroke")
plt.show()

In [ ]:
# bmi vs stroke (boxplot)
plt.figure(figsize=(6,4))
sns.boxplot(x="stroke", y="bmi", data=df)
plt.title("BMI by Stroke")
plt.show()

###Avg glucose & BMI boxplots: Higher median glucose in stroke patients suggests metabolic link. For BMI, check whether stroke patients have higher/lower medians — sometimes underweight or very high BMI show different risks.

In [ ]:
# smoking status vs stroke
plt.figure(figsize=(8,4))
ax = sns.countplot(x="smoking_status", hue="stroke", data=df)
add_percent(ax, df, "smoking_status", "stroke")
plt.xticks(rotation=30)
plt.title("Smoking Status vs Stroke (%)")
plt.show()

###Smoking status: Look for elevated stroke proportions among current/former smokers. Again watch for small sample sizes in categories like "Unknown" or "never smoked".

###5 — Missing values & imputation
1. missing_values_and_imputation.py
2.  Check missing values and impute BMI with mean (as you did)

In [ ]:
# Impute BMI with mean (documented)
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())
print("\nAfter imputing BMI, missing values:")
display(df.isnull().sum())

###Interpretation:

* If BMI had missing values only, mean imputation is simple but can underestimate variance. Alternatives: median, KNN imputation, or model-based imputation. Use mean if distribution is symmetric; if skewed prefer median.

###6 — Encoding categorical variables
* Map and one-hot encode categorical variables. Keep dtype=int for clarity.

In [ ]:
# Map small categories to numeric
df['gender'] = df['gender'].map({'Male':1, 'Female':0, 'Other':2})
df['ever_married'] = df['ever_married'].map({'Yes':1, 'No':0})

In [ ]:
# One-hot encode multi-category strings
df = pd.get_dummies(df, columns=['work_type','Residence_type',\
 'smoking_status'], drop_first=False, dtype=int)

In [ ]:
# Drop id column
if 'id' in df.columns:
  df.drop('id', axis=1, inplace=True)
display(df.head())

###Notes:

* You used drop_first=False — that's fine. If you prefer fewer columns and don't mind collinearity for tree models, keep them as is. For linear models, one-hot with drop_first=True reduces linear dependence; but since you will scale and perhaps regularize, either is acceptable

###7 — Outlier detection

In [ ]:
def detect_outliers_iqr(df, column):
    """
    Print count of IQR outliers and the acceptable range.
    Returns DataFrame of outlier rows.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower) | (df[column] > upper)]
    print(f"{column}: outliers = {len(outliers)}")
    print(f"Acceptable range: {lower:.2f} to {upper:.2f}")
    return outliers

out_glucose = detect_outliers_iqr(df, 'avg_glucose_level')
out_bmi = detect_outliers_iqr(df, 'bmi')

In [ ]:
# Show a few outlier rows for inspection
display(out_glucose.head())
display(out_bmi.head())

###Interpretation & action suggestions:

1. Outliers may be true extreme physiological measurements or data errors. Investigate a few rows (age, stroke label) before dropping.

2. For modeling, robust scaling or winsorization are alternatives to dropping outliers.

###8 — Split, scale, and resample (SMOTE)


In [ ]:
# Prepare features and target, split, then scale using train-fit only (avoid data leakage).
features = df.drop('stroke', axis=1)
target = df['stroke']

In [ ]:
# Train-test split BEFORE scaling to avoid leakage
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2\
, random_state=RANDOM_STATE, stratify=target)

In [ ]:
# Standardize numeric features (fit on train, transform train and test)
numeric_cols = ['age', 'avg_glucose_level', 'bmi']
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [ ]:
# Handle class imbalance with SMOTE on the training set ONLY
sm = SMOTE(random_state=RANDOM_STATE)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [ ]:
print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", y_train_res.value_counts())

###Why this order?
* You must fit the scaler on train set only to avoid leaking distributional information from test set into training (data leakage). Applying SMOTE and scaling should be done after splitting, and SMOTE only on the training set.

#9 — Evaluation functio9 — Evaluation function

In [ ]:
# 9_evaluation_function.py
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    """
    Fit the model on (X_train, y_train), then evaluate on (X_test, y_test).
    Print classification report, accuracy, AUC. Plot confusion matrix
     and ROC curve.
    """
    # Train
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    # Some classifiers may not have predict_proba (e.g., certain SVMs)
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
    else:
        # fallback to decision_function scaled to [0,1] if available
        if hasattr(model, "decision_function"):
            scores = model.decision_function(X_test)
            # scale to 0..1
            y_prob = (scores - scores.min()) / (scores.max() -\
            scores.min() + 1e-8)
        else:
            y_prob = np.zeros(len(y_test))  # not ideal, but prevents crashes

    # Reports
    print("\n" + "="*60)
    print(f"Result: {model_name}")
    print("="*60 + "\n")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=3))
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc*100:.2f}%")

    # AUC may fail if y_prob constant; guard it
    try:
        auc = roc_auc_score(y_test, y_prob)
        print(f"AUC: {auc*100:.2f}%")
    except Exception as e:
        print("AUC: Could not compute (possibly constant predictions).", e)
    print('\n')
    # Confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()
    print("\n")
    # ROC curve plot (if probabilities available)
    if np.unique(y_prob).size > 1:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure(figsize=(5,4))
        plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y_test, y_prob):.3f}")
        plt.plot([0,1], [0,1], '--', color='gray')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve - {model_name}")
        plt.legend()
        plt.show()
    else:
        print("ROC curve: skipped (no probability variation).")


#Note:
*  This function trains the model internally. Call it with the resampled training data and the untouched test set.

##10 — Train & evaluate models

In [ ]:
# Logistic Regression
log_model = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
evaluate_model(log_model, X_train_res, y_train_res, X_test, y_test, "Logistic Regression")

##Accuracy (81.9%) indicates that the model correctly classified about 82% of all samples.

However, Recall for class 1 (stroke = 0.700) shows that it detected 70% of stroke cases — quite high for an imbalanced dataset.

The Precision (0.171) for stroke is low, meaning it produces many false positives.

AUC = 82.03% reflects good overall discrimination ability between stroke and non-stroke cases.

Interpretation:
The Logistic Regression model is relatively balanced — it sacrifices some precision to detect more stroke cases, which is acceptable in a healthcare context where missing a stroke case is more dangerous than a false

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
evaluate_model(rf_model, X_train_res, y_train_res, X_test, y_test, "Random Forest")

##Accuracy (90.6%) looks high, but it’s misleading because the dataset is imbalanced.

Recall for class 1 (0.160) means the model only detects 16% of actual stroke cases — very poor sensitivity.

Precision (0.129) is also low, indicating poor detection of true stroke patients.

AUC = 78.07% confirms moderate discrimination power.

Interpretation:
The Random Forest performs well for the majority class (non-stroke) but fails to detect stroke cases effectively — not suitable for medical diagnosis as is.

In [ ]:
# XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE)
evaluate_model(xgb_model, X_train_res, y_train_res, X_test, y_test, "XGBoost")

##Similar to Random Forest with Accuracy ≈ 90.7%,
but Recall (0.180) and Precision (0.143) show slightly better performance on stroke detection.

AUC = 78.34% — slightly higher than Random Forest, meaning marginally better distinction between classes.

Interpretation:
XGBoost improves slightly over Random Forest, but still struggles with the minority (stroke) class. Further tuning or resampling (e.g., SMOTE during training) is needed.

### Model Evaluation Summary
Among the three models tested (Logistic Regression, Random Forest, XGBoost),  
**Logistic Regression** achieved the best recall (0.70) and strong AUC (82%),  
making it the most suitable model for detecting stroke cases.  
While Random Forest and XGBoost had higher accuracy, their recall for stroke cases was poor,  
which is critical in medical predictions.


#11 —Save the trained model and preprocessing objects


In [ ]:
import joblib # import librarary to use it save the project

In [ ]:
#Save the trained Logistic Regression model as a binary file (.pkl)
# This allows us to reload and use the model later without retraining.
joblib.dump(log_model, 'best_model_logistic_regression.pkl')

In [ ]:
# Save the feature names used during model training
# Ensures that new data passed to the model will have the same structure.
joblib.dump(list(features.columns), 'model_features.pkl')

In [ ]:
# Save the fitted scaler to apply the exact same scaling on future data
# This is critical to maintain consistency between training and inference.
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
print("✅ Model, features, and scaler saved successfully!")

In [ ]:
# Import the Colab file utility
from google.colab import files

In [ ]:
# Download each saved file to your laptop
# These files can later be loaded in VS Code or any Python environment.
files.download('best_model_logistic_regression.pkl')
files.download('model_features.pkl')
files.download('scaler.pkl')

print("📂 All files downloaded successfully to your computer!")

In [ ]:
#Stroke Prediction Web App using Gradio
import gradio as gr

In [ ]:
# =====================================================
# Load Trained Model, Scaler, and Feature Names
# =====================================================
model = joblib.load("best_model_logistic_regression.pkl")   # Trained ML model
features = joblib.load("model_features.pkl")                # List of features used during training
scaler = joblib.load("scaler.pkl")                          # Scaler used for normalization)

In [ ]:
import gradio as gr
import joblib
import pandas as pd

# =========================
# Load model, scaler, and features
# =========================
model = joblib.load("best_model_logistic_regression.pkl")
scaler = joblib.load("scaler.pkl")
features = joblib.load("model_features.pkl")

# =========================
# Prediction Function
# =========================
def predict_stroke(age, hypertension, heart_disease, avg_glucose_level, bmi,
                   gender, ever_married, work_type, Residence_type, smoking_status):

    # Build input DataFrame
    data = pd.DataFrame([{
        "age": age,
        "hypertension": hypertension,
        "heart_disease": heart_disease,
        "avg_glucose_level": avg_glucose_level,
        "bmi": bmi,
        "gender": gender,
        "ever_married": ever_married,
        "work_type": work_type,
        "Residence_type": Residence_type,
        "smoking_status": smoking_status
    }])

    # One-hot encode
    data_ready = pd.get_dummies(data)

    # Reindex to match training features
    data_ready = data_ready.reindex(columns=features, fill_value=0)

    # Scale numeric features
    numeric_cols = ["age", "avg_glucose_level", "bmi"]
    data_ready[numeric_cols] = scaler.transform(data_ready[numeric_cols])

    # Make prediction
    pred_prob = model.predict_proba(data_ready)[0][1]
    prediction = "Stroke Risk Detected ⚠️" if pred_prob > 0.5 else "No Stroke Risk ✅"

    return f"**{prediction}**\n\nProbability: {pred_prob:.2%}"

# =========================
# Gradio UI
# =========================
with gr.Blocks(title="Stroke Risk Predictor - Designed by Islam Abdul Rahim Mohamed") as demo:


    gr.Markdown("""
    <h2 style="text-align:center;">🧠 Stroke Prediction Web App</h2>
    <p style="text-align:center;">Provide patient information below to estimate stroke risk.</p>
    """, elem_id="header")

    with gr.Row():
        with gr.Column():
            age = gr.Number(label="Age", value=45)
            hypertension = gr.Dropdown(["0", "1"], label="Hypertension (0 = No, 1 = Yes)", value="0")
            heart_disease = gr.Dropdown(["0", "1"], label="Heart Disease (0 = No, 1 = Yes)", value="0")
            avg_glucose_level = gr.Number(label="Average Glucose Level", value=90.0)
            bmi = gr.Number(label="BMI", value=25.0)

        with gr.Column():
            gender = gr.Dropdown(["Male", "Female", "Other"], label="Gender", value="Male")
            ever_married = gr.Dropdown(["Yes", "No"], label="Ever Married", value="No")
            work_type = gr.Dropdown(["Private", "Self-employed", "Govt_job", "Children", "Never_worked"], label="Work Type", value="Private")
            Residence_type = gr.Dropdown(["Urban", "Rural"], label="Residence Type", value="Urban")
            smoking_status = gr.Dropdown(["formerly smoked", "never smoked", "smokes", "Unknown"], label="Smoking Status", value="never smoked")

    predict_button = gr.Button("🔍 Predict Stroke Risk", variant="primary")
    output = gr.Markdown(label="Result")

    predict_button.click(
        predict_stroke,
        inputs=[age, hypertension, heart_disease, avg_glucose_level, bmi,
                gender, ever_married, work_type, Residence_type, smoking_status],
        outputs=output
    )

    # Footer with signature
    gr.Markdown("""
    ---
    <p style="text-align:center; font-size:14px; color:gray;">
    🩺 <b>Model:</b> Logistic Regression &nbsp;&nbsp;|&nbsp;&nbsp; 📦 <b>Built with:</b> Scikit-learn & Gradio
    </p>
    <p style="text-align:center; font-family:'Brush Script MT', cursive; font-size:20px; color:#1a202c;">
    Designed by <span style="color:#2b6cb0;">Islam Abdul Rahim Mohamed</span>
    </p>
    """)

# =========================
# Launch the app
# =========================
demo.launch()


###### Model Inference (Prediction)
The saved model can be reloaded and used to predict stroke risk for new patients.


###### Technical Notes
- SMOTE was applied **only on training data** to prevent data leakage.  
- Test data was kept untouched for fair evaluation.  
- Models were evaluated using Accuracy, Recall, Precision, F1-score, and AUC.


### Discussion & Future Work
The Logistic Regression model performed best, but there is room for improvement.  
Future work could include:
- Hyperparameter tuning using GridSearchCV.  
- Feature importance analysis to understand which health factors matter most.  
- Trying advanced ensemble methods or neural networks.  
- Deploying the model as a web or mobile app for clinical screening.


##Project Summary
This project explored stroke prediction using multiple machine learning models.  
After data cleaning, preprocessing, and resampling with SMOTE,  
Logistic Regression achieved the best performance (AUC = 82%, Recall = 0.70).  
The model was saved for deployment and future analysis.
